In [1]:
API_Key = "NZuKNoxEadqDrXu1uaGhrO0FD0b8RTErPk4r42Ra"

import requests

url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-08&api_key={API_Key}"

response = requests.get(url)

data = response.json()

details = data["near_earth_objects"]

asteroids_data = []
target = 10000

url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2024-01-01&end_date=2024-01-08&api_key={API_Key}"

while len(asteroids_data) < target:
    response = requests.get(url)
    data = response.json()
    details = data["near_earth_objects"]
    for date, asteroids in details.items():
        for asteroid in asteroids:
            asteroids_data.append(dict(id= asteroid['id'],
                                   neo_reference_id= asteroid['neo_reference_id'],
                                   name= asteroid['name'],
                                   absolute_magnitude_h= asteroid['absolute_magnitude_h'],
                                   estimated_diameter_min_km= asteroid['estimated_diameter']['kilometers']['estimated_diameter_min'],
                                   estimated_diameter_max_km= asteroid['estimated_diameter']['kilometers']['estimated_diameter_max'],
                                   is_potentially_hazardous_asteroid= asteroid['is_potentially_hazardous_asteroid'],
                                   close_approach_date= asteroid['close_approach_data'][0]['close_approach_date'],
                                   relative_velocity_kmph= float(asteroid['close_approach_data'][0]['relative_velocity']['kilometers_per_hour']),
                                   astronomical= float(asteroid['close_approach_data'][0]['miss_distance']['astronomical']),
                                   miss_distance_km= float(asteroid['close_approach_data'][0]['miss_distance']['kilometers']),
                                   miss_distance_lunar= float(asteroid['close_approach_data'][0]['miss_distance']['lunar']),
                                   orbiting_body= asteroid['close_approach_data'][0]['orbiting_body']))
            if len(asteroids_data) >= target:
                break  # Break the inner loop once the target is reached
        if len(asteroids_data) >= target:
            break # Break the outer loop once the target is reached
    if len(asteroids_data) >= target:
            break # Break the while loop once the target is reached
    url = data['links']['next']


In [2]:
asteroids_data[0]

{'id': '2415949',
 'neo_reference_id': '2415949',
 'name': '415949 (2001 XY10)',
 'absolute_magnitude_h': 19.37,
 'estimated_diameter_min_km': 0.3552670883,
 'estimated_diameter_max_km': 0.7944013596,
 'is_potentially_hazardous_asteroid': False,
 'close_approach_date': '2024-01-02',
 'relative_velocity_kmph': 57205.8951204341,
 'astronomical': 0.3372535274,
 'miss_distance_km': 50452409.349026635,
 'miss_distance_lunar': 131.1916221586,
 'orbiting_body': 'Earth'}

In [3]:
from datetime import datetime
for obj in asteroids_data:
    # Convert date string to datetime.date
    obj['close_approach_date'] = datetime.strptime(obj['close_approach_date'], "%Y-%m-%d").date()

In [4]:
import sqlite3

# Connect to a database file (creates it if it doesn't exist)
conn = sqlite3.connect('NASA_NEO_new')

cursor = conn.cursor()

In [5]:
import pandas as pd

In [6]:
cursor.execute('''CREATE TABLE Asteroids(
    id INT,
    name TEXT,
    absolute_magnitude_h FLOAT,
    estimated_diameter_min_km FLOAT,
    estimated_diameter_max_km FLOAT,
    is_potentially_hazardous_asteroid BOOLEAN
)''')

In [7]:
cursor.execute('''CREATE TABLE Close_approach(
    neo_reference_id INT,
    close_approach_date DATE,
    relative_velocity_kmph FLOAT,
    astronomical FLOAT,
    miss_distance_km FLOAT,
    miss_distance_lunar FLOAT,
    orbiting_body VARCHAR
)''')


In [8]:
for obj in asteroids_data:
  cursor.execute('''INSERT INTO Asteroids(id,
                                          name,
                                          absolute_magnitude_h,
                                          estimated_diameter_min_km,
                                          estimated_diameter_max_km,
                                          is_potentially_hazardous_asteroid)
  VALUES(?, ?, ?, ?, ?, ?)''',(obj['id'],
                               obj['name'],
                               obj['absolute_magnitude_h'],
                               obj['estimated_diameter_min_km'],
                               obj['estimated_diameter_max_km'],
                               obj['is_potentially_hazardous_asteroid']))

  cursor.execute('''INSERT INTO Close_approach(neo_reference_id,
                                              close_approach_date,
                                              relative_velocity_kmph,
                                              astronomical,
                                              miss_distance_km,
                                              miss_distance_lunar,
                                              orbiting_body)
  VALUES(?, ?, ?, ?, ?, ?, ?)''',(obj['neo_reference_id'],
                                  obj['close_approach_date'],
                                  obj['relative_velocity_kmph'],
                                  obj['astronomical'],
                                  obj['miss_distance_km'],
                                  obj['miss_distance_lunar'],
                                  obj['orbiting_body']))

In [9]:
conn.commit()

In [10]:
#1.Count how many times each asteroid has approached Earth
query1 = '''SELECT neo_reference_id,
           COUNT(*) As Approach_count
           FROM close_approach
           GROUP BY neo_reference_id
           ORDER BY Approach_count DESC'''
pd.read_sql_query(query1, conn)

,neo_reference_id,Approach_count
0,54526031,5
1,54471061,5
2,54342587,5
3,54293876,5
4,3825135,5
...,...,...
7497,2004034,1
7498,2002340,1
7499,2002063,1
7500,2001685,1


In [11]:
#2. Average velocity of each asteroid over multiple approaches
query2 = '''
SELECT
    ca.neo_reference_id,
    a.name,
    ROUND(AVG(ca.relative_velocity_kmph), 2) AS avg_velocity_kmph
FROM
    close_approach ca
JOIN
    asteroids a ON ca.neo_reference_id = a.id
GROUP BY
    ca.neo_reference_id
ORDER BY
    avg_velocity_kmph DESC;
'''

pd.read_sql_query(query2, conn)

,neo_reference_id,name,avg_velocity_kmph
0,54277079,(2022 HB4),190513.42
1,54392072,(2023 TS3),173071.83
2,3803884,(2018 GG5),161520.90
3,3790060,(2017 WP28),158279.86
4,3425295,(2008 QB),154336.41
...,...,...,...
7497,54471067,(2024 RA16),2986.38
7498,54459240,(2024 NS2),2150.00
7499,3798020,(2018 BW5),1909.58
7500,54440316,(2024 GB15),1610.57


In [12]:
#3.List top 10 fastest asteroids
query3 = '''
SELECT
    ca.neo_reference_id,
    a.name,
    MAX(ca.relative_velocity_kmph) AS max_velocity_kmph
FROM
    close_approach ca
JOIN
    asteroids a ON ca.neo_reference_id = a.id
GROUP BY
    ca.neo_reference_id
ORDER BY
    max_velocity_kmph DESC
LIMIT 10;
'''
pd.read_sql_query(query3, conn)

,neo_reference_id,name,max_velocity_kmph
0,54277079,(2022 HB4),190513.420686
1,54392072,(2023 TS3),173071.830081
2,3803884,(2018 GG5),161520.897925
3,3790060,(2017 WP28),158279.864701
4,3425295,(2008 QB),154336.407297
5,3092220,(1999 GR6),152218.220518
6,3588900,(2011 WD39),151520.395439
7,54448599,(2024 MH),150599.454097
8,54111253,(2021 CQ3),150583.019419
9,3383901,(2007 RT1),149261.278634


In [13]:
#4.Find potentially hazardous asteroids that have approached Earth more than 3 times
query4 = '''
SELECT
    a.name,
    ca.neo_reference_id,
    COUNT(*) AS approach_count
FROM
    close_approach ca
JOIN
    asteroids a ON ca.neo_reference_id = a.id
WHERE
    a.is_potentially_hazardous_asteroid = 1
GROUP BY
    ca.neo_reference_id
HAVING
    approach_count > 3
ORDER BY
    approach_count DESC;
'''
pd.read_sql_query(query4, conn)

,name,neo_reference_id,approach_count
0,(2010 VD72),3551067,16
1,(2025 AK2),54514905,9
2,(2023 QL7),54378880,9
3,(2022 US21),54322868,9
4,(2018 MY6),3825481,9
...,...,...,...
86,163026 (2001 XR30),2163026,4
87,162421 (2000 ET70),2162421,4
88,162361 (2000 AF6),2162361,4
89,101955 Bennu (1999 RQ36),2101955,4


In [14]:
#5.Find the month with the most asteroid approaches
query5 = '''
SELECT
    STRFTIME('%Y-%m', close_approach_date) AS month,
    COUNT(*) AS total_approaches
FROM
    close_approach
GROUP BY
    month
ORDER BY
    total_approaches DESC
LIMIT 1;
'''
pd.read_sql_query(query5, conn)



,month,total_approaches
0,2024-10,908


In [15]:
#6.Get the asteroid with the fastest ever approach speed
query6 = '''
SELECT
    a.name,
    ca.neo_reference_id,
    ca.relative_velocity_kmph,
    ca.close_approach_date
FROM
    close_approach ca
JOIN
    asteroids a ON ca.neo_reference_id = a.id
ORDER BY
    ca.relative_velocity_kmph DESC
LIMIT 1;
'''
pd.read_sql_query(query6, conn)

,name,neo_reference_id,relative_velocity_kmph,close_approach_date
0,(2022 HB4),54277079,190513.420686,2025-04-15


In [16]:
#7.Sort asteroids by maximum estimated diameter (descending)
query7 = '''
SELECT
    name,
    estimated_diameter_max_km
FROM
    asteroids
ORDER BY
    estimated_diameter_max_km DESC;
'''
pd.read_sql_query(query7, conn)

,name,estimated_diameter_max_km
0,887 Alinda (A918 AA),10.233872
1,1685 Toro (1948 OA),8.280184
2,66146 (1998 TU3),7.980681
3,5660 (1974 MA),4.989304
4,533671 (2014 LJ21),3.926811
...,...,...
9995,(2024 YW8),0.002208
9996,(2023 RS),0.002042
9997,(2023 RS),0.002042
9998,(2024 RC42),0.001986


In [17]:
#8.An asteroid whose closest approach is getting nearer over time
query8 = '''
SELECT
    a.name,
    c.close_approach_date,
    c.miss_distance_km
FROM
    asteroids a
JOIN
    close_approach c
ON
    a.id = c.neo_reference_id
WHERE
    a.id IN (
        SELECT neo_reference_id
        FROM close_approach
        GROUP BY neo_reference_id
        HAVING COUNT(*) > 1
    )
ORDER BY
    a.name,
    c.close_approach_date;
'''
pd.read_sql_query(query8, conn)

,name,close_approach_date,miss_distance_km
0,(1993 KA),2024-03-17,4.252127e+07
1,(1993 KA),2024-03-17,4.252127e+07
2,(1993 KA),2024-08-11,4.992568e+07
3,(1993 KA),2024-08-11,4.992568e+07
4,(1994 GL),2024-01-11,6.839243e+07
...,...,...,...
10571,806899 (2017 BO31),2025-01-27,1.687315e+07
10572,9058 (1992 JB),2025-04-21,1.529077e+07
10573,9058 (1992 JB),2025-04-21,1.529077e+07
10574,9058 (1992 JB),2025-04-21,1.529077e+07


In [18]:
#9.Display the name of each asteroid along with the date and miss distance of its closest approach to Earth.
query9 = '''
SELECT
    a.name,
    c.close_approach_date,
    c.miss_distance_km
FROM
    asteroids a
JOIN
    close_approach c
ON
    a.id = c.neo_reference_id
ORDER BY
    c.miss_distance_km ASC;
'''
pd.read_sql_query(query9, conn)

,name,close_approach_date,miss_distance_km
0,(2024 XA),2024-12-01,7.726027e+03
1,(2024 LH1),2024-06-06,8.098256e+03
2,(2024 LH1),2024-06-06,8.098256e+03
3,(2024 LH1),2024-06-06,8.098256e+03
4,(2024 UG9),2024-10-30,8.849866e+03
...,...,...,...
16011,(2020 YH),2025-01-13,7.477857e+07
16012,(2020 YH),2025-01-13,7.477857e+07
16013,(2023 GF),2025-04-10,7.478050e+07
16014,(2021 UQ3),2025-01-28,7.478989e+07


In [19]:
#10.List names of asteroids that approached Earth with velocity > 50,000 km/h

query10 = '''
SELECT
    a.name,
    c.relative_velocity_kmph
FROM
    asteroids a
JOIN
    close_approach c
ON
    a.id = c.neo_reference_id
WHERE
    c.relative_velocity_kmph > 50000
ORDER BY
    c.relative_velocity_kmph DESC;
'''
pd.read_sql_query(query10, conn)

,name,relative_velocity_kmph
0,(2022 HB4),190513.420686
1,(2023 TS3),173071.830081
2,(2018 GG5),161520.897925
3,(2017 WP28),158279.864701
4,(2008 QB),154336.407297
...,...,...
5451,(2021 GN1),50027.279641
5452,(2021 GN1),50027.279641
5453,(2021 GN1),50027.279641
5454,512245 (2016 AU8),50022.866129


In [20]:
#11.Count how many approaches happened per month

query11 = '''
SELECT
    strftime('%Y-%m', close_approach_date) AS year_month,
    COUNT(*) AS approach_count
FROM
    close_approach
GROUP BY
    year_month
ORDER BY
    year_month;
'''
pd.read_sql_query(query11, conn)

,year_month,approach_count
0,2024-01,628
1,2024-02,621
2,2024-03,637
3,2024-04,712
4,2024-05,563
5,2024-06,492
6,2024-07,470
7,2024-08,591
8,2024-09,841
9,2024-10,908


In [21]:
#12.Find asteroid with the highest brightness (lowest magnitude value)
query12 = '''
SELECT
    name,
    absolute_magnitude_h
FROM
    asteroids
ORDER BY
    absolute_magnitude_h ASC
LIMIT 1;
'''
pd.read_sql_query(query12, conn)

,name,absolute_magnitude_h
0,887 Alinda (A918 AA),13.82


In [22]:
#13.Get number of hazardous vs non-hazardous asteroids
query13 ='''
SELECT
    is_potentially_hazardous_asteroid AS hazardous_status,
    COUNT(*) AS count
FROM
    asteroids
GROUP BY
    is_potentially_hazardous_asteroid;
'''
pd.read_sql_query(query13, conn)



,hazardous_status,count
0,0,9504
1,1,496


In [23]:
#14.Find asteroids that passed closer than the Moon (lesser than 1 LD), along with their close approach date and distance.(LD"Lunar distance" = 384,400 km)
query14 = '''
SELECT
    a.name,
    c.close_approach_date,
    c.miss_distance_lunar
FROM
    asteroids a
JOIN
    close_approach c
ON
    a.id = c.neo_reference_id
WHERE
    c.miss_distance_lunar < 1
ORDER BY
    c.miss_distance_lunar ASC;
'''
pd.read_sql_query(query14, conn)

,name,close_approach_date,miss_distance_lunar
0,(2024 XA),2024-12-01,0.020090
1,(2024 LH1),2024-06-06,0.021058
2,(2024 LH1),2024-06-06,0.021058
3,(2024 LH1),2024-06-06,0.021058
4,(2024 UG9),2024-10-30,0.023012
...,...,...,...
349,(2024 RR8),2024-09-02,0.979981
350,(2024 RR8),2024-09-02,0.979981
351,(2024 RR8),2024-09-02,0.979981
352,(2024 RR8),2024-09-02,0.979981


In [24]:
#15.Find asteroids that came within 0.05 AU(astronomical distance)
query15 = '''
SELECT
    a.name,
    c.close_approach_date,
    c.astronomical
FROM
    asteroids a
JOIN
    close_approach c
ON
    a.id = c.neo_reference_id
WHERE
    c.astronomical < 0.05
ORDER BY
    c.astronomical ASC;
'''
pd.read_sql_query(query15, conn)

,name,close_approach_date,astronomical
0,(2024 XA),2024-12-01,0.000052
1,(2024 LH1),2024-06-06,0.000054
2,(2024 LH1),2024-06-06,0.000054
3,(2024 LH1),2024-06-06,0.000054
4,(2024 UG9),2024-10-30,0.000059
...,...,...,...
3570,(2024 UF2),2024-11-04,0.049892
3571,(2024 QJ1),2024-07-27,0.049906
3572,(2006 GB1),2024-10-08,0.049947
3573,(2024 FV),2024-03-16,0.049956


In [25]:
#16. Top 10 Largest Asteroids (by estimated max diameter)

query16 = '''
SELECT
    name,
    estimated_diameter_max_km
FROM
    asteroids
ORDER BY
    estimated_diameter_max_km DESC
LIMIT 10;
'''
pd.read_sql_query(query16, conn)

,name,estimated_diameter_max_km
0,887 Alinda (A918 AA),10.233872
1,1685 Toro (1948 OA),8.280184
2,66146 (1998 TU3),7.980681
3,5660 (1974 MA),4.989304
4,533671 (2014 LJ21),3.926811
5,415029 (2011 UL21),3.872933
6,66008 (1998 QH2),3.855138
7,88188 (2000 XH44),3.614431
8,16816 (1997 UF9),3.373184
9,36183 (1999 TX16),3.266179


In [26]:
#17. Closest approach per asteroid (their nearest pass)

query17 = '''
SELECT
    a.name,
    MIN(c.miss_distance_km) AS closest_approach_km
FROM
    asteroids a
JOIN
    close_approach c ON a.id = c.neo_reference_id
GROUP BY
    a.id
ORDER BY
    closest_approach_km ASC;
'''
pd.read_sql_query(query17, conn)

,name,closest_approach_km
0,(2024 XA),7.726027e+03
1,(2024 LH1),8.098256e+03
2,(2024 UG9),8.849866e+03
3,(2025 BP6),9.711954e+03
4,(2024 HA),1.555513e+04
...,...,...
7497,(2019 LX),7.476755e+07
7498,(2020 YH),7.477857e+07
7499,(2023 GF),7.478050e+07
7500,(2021 UQ3),7.478989e+07


In [27]:
#18.  Monthly trend of potentially hazardous approaches

query18 = '''
SELECT
    strftime('%Y-%m', c.close_approach_date) AS year_month,
    COUNT(*) AS hazardous_approach_count
FROM
    asteroids a
JOIN
    close_approach c ON a.id = c.neo_reference_id
WHERE
    a.is_potentially_hazardous_asteroid = 1
GROUP BY
    year_month
ORDER BY
    year_month;
'''
pd.read_sql_query(query18, conn)

,year_month,hazardous_approach_count
0,2024-01,65
1,2024-02,54
2,2024-03,45
3,2024-04,64
4,2024-05,35
5,2024-06,52
6,2024-07,69
7,2024-08,42
8,2024-09,68
9,2024-10,45


In [28]:
#19. Asteroids that have multiple close approaches
query19 = '''
SELECT
    a.name,
    COUNT(*) AS approach_count
FROM
    asteroids a
JOIN
    close_approach c ON a.id = c.neo_reference_id
GROUP BY
    a.id
HAVING
    approach_count > 1
ORDER BY
    approach_count DESC;
'''
pd.read_sql_query(query19, conn)


,name,approach_count
0,(2025 FJ22),25
1,(2024 RT15),25
2,(2023 EF),25
3,(2020 AQ24),25
4,(2018 LV2),25
...,...,...
2057,162361 (2000 AF6),4
2058,101955 Bennu (1999 RQ36),4
2059,42286 (2001 TN41),4
2060,26663 (2000 XK47),4


In [29]:
#20. Fastest 5 Asteroids during approach
query20 = '''
SELECT
    a.name,
    c.relative_velocity_kmph
FROM
    asteroids a
JOIN
    close_approach c ON a.id = c.neo_reference_id
ORDER BY
    c.relative_velocity_kmph DESC
LIMIT 5;
'''
pd.read_sql_query(query20, conn)

,name,relative_velocity_kmph
0,(2022 HB4),190513.420686
1,(2023 TS3),173071.830081
2,(2018 GG5),161520.897925
3,(2017 WP28),158279.864701
4,(2008 QB),154336.407297
